In [57]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report

In [58]:
df = pd.read_csv("du_lieu_dumies.csv")

In [59]:
df.head()

,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,7590-VHVEG,Female,No,Yes,No,1,No,No phone service,DSL,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [60]:
df.drop('customerID',axis=1, inplace=True)
df.drop('Unnamed: 0',axis=1, inplace=True)

In [61]:
df = pd.get_dummies(df)
df.head()

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_No,SeniorCitizen_Yes,Partner_No,Partner_Yes,Dependents_No,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,1,29.85,29.85,1,0,1,0,0,1,1,...,0,0,0,1,0,0,1,0,1,0
1,34,56.95,1889.50,0,1,1,0,1,0,1,...,1,0,1,0,0,0,0,1,1,0
2,2,53.85,108.15,0,1,1,0,1,0,1,...,0,0,0,1,0,0,0,1,0,1
3,45,42.30,1840.75,0,1,1,0,1,0,1,...,1,0,1,0,1,0,0,0,1,0
4,2,70.70,151.65,1,0,1,0,1,0,1,...,0,0,0,1,0,0,1,0,0,1


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 48 columns):
tenure                                     7032 non-null int64
MonthlyCharges                             7032 non-null float64
TotalCharges                               7032 non-null float64
gender_Female                              7032 non-null uint8
gender_Male                                7032 non-null uint8
SeniorCitizen_No                           7032 non-null uint8
SeniorCitizen_Yes                          7032 non-null uint8
Partner_No                                 7032 non-null uint8
Partner_Yes                                7032 non-null uint8
Dependents_No                              7032 non-null uint8
Dependents_Yes                             7032 non-null uint8
PhoneService_No                            7032 non-null uint8
PhoneService_Yes                           7032 non-null uint8
MultipleLines_No                           7032 non-null uint8
Multipl

In [63]:
df.drop('PaymentMethod_Credit card (automatic)', axis=1, inplace=True)
df.drop('PaymentMethod_Electronic check', axis=1, inplace=True)
df.drop('PaymentMethod_Mailed check', axis=1, inplace=True)

In [64]:
X = df.drop("PaymentMethod_Bank transfer (automatic)", axis=1)
y = df["PaymentMethod_Bank transfer (automatic)"]

In [65]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(C=0.1, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='ovr', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=1e-06, verbose=0,
                   warm_start=False)

In [66]:
from sklearn.model_selection import KFold
def run_kfold(clf_lr):
    kf = KFold(n_splits =10, random_state =42, shuffle = True)
    outcomes1 = []
    outcomes2 = []
    outcomes3 = []
    outcomes4 = []
    fold = 0
    for train_index, test_index in  kf.split(df):
        fold += 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf_lr.fit(X_train, y_train)
        Yhat_test = clf_lr.predict(X_test)
        precision = precision_score(y_test, Yhat_test,average='micro')
        recall = recall_score(y_test, Yhat_test,average='micro')
        f1 = f1_score(y_test, Yhat_test,average='micro')
        accuracy = accuracy_score(y_test, Yhat_test)
        outcomes1.append(precision)
        outcomes2.append(recall)
        outcomes3.append(f1)
        outcomes4.append(accuracy)
        print("Fold {0} precision: {1}".format(fold, precision))
        print("Fold {0} recall: {1}".format(fold, recall))
        print("Fold {0} f1: {1}".format(fold, f1))
        print("Fold {0} accuracy: {1}".format(fold, accuracy))
    mean_outcome1 = np.mean(outcomes1)
    mean_outcome2 = np.mean(outcomes2)
    mean_outcome3 = np.mean(outcomes3)    
    mean_outcome4 = np.mean(outcomes4)
    print("Mean precision: {0}".format(mean_outcome1))
    print("Mean recall: {0}".format(mean_outcome2))
    print("Mean f1: {0}".format(mean_outcome3))
    print("Mean Accuracy: {0}".format(mean_outcome4)) 
run_kfold(clf_lr)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fold 1 precision: 0.7457386363636364
Fold 1 recall: 0.7457386363636364
Fold 1 f1: 0.7457386363636362
Fold 1 accuracy: 0.7457386363636364
Fold 2 precision: 0.7684659090909091

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



Fold 2 recall: 0.7684659090909091
Fold 2 f1: 0.7684659090909091
Fold 2 accuracy: 0.7684659090909091


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...


Fold 8 recall: 0.7937411095305832
Fold 8 f1: 0.7937411095305833
Fold 8 accuracy: 0.7937411095305832


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fold 9 precision: 0.7795163584637269
Fold 9 recall: 0.7795163584637269
Fold 9 f1: 0.7795163584637268
Fold 9 accuracy: 0.7795163584637269
Fold 10 precision: 0.7923186344238976

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



Fold 10 recall: 0.7923186344238976
Fold 10 f1: 0.7923186344238976
Fold 10 accuracy: 0.7923186344238976
Mean precision: 0.7807234417431785
Mean recall: 0.7807234417431785
Mean f1: 0.7807234417431785
Mean Accuracy: 0.7807234417431785
